# Goh Jun Hao (G2102327H)
# AI6126 Project 2: Blind Face Super Resolution

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Install MMEditing

In [ ]:
# Install openmim for installation of mmcv-full
!pip install openmim

# Install mmcv-full thus we could use CUDA operators
!mim install mmcv-full

# Clone MMEditing
!rm -rf mmediting
!git clone https://github.com/open-mmlab/mmediting.git
%cd mmediting

# Install MMEditing
!pip install -v -e .

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMEditing installation
import mmedit
print(mmedit.__version__)

# Generate ann.txt

In [5]:
PATH='/content/gdrive/MyDrive/data/FFHQ/'

In [6]:
import os.path as osp
import mmcv

data_root = '/content/gdrive/MyDrive/data/FFHQ/'
train_img_dir = 'train/GT'
# train_ann_dir = 'train/train_mask'
val_img_dir = 'val/GT'
val_ann_dir = 'val/LQ'
# test_img_dir = 'test/test_image'
test_ann_dir = 'test/LQ'
# split train/val set randomly
import glob
gt_paths = sorted(glob.glob(PATH+train_img_dir+'/*.png'))
# print(gt_paths)
with open(PATH+train_img_dir+'/training_ann.txt', 'w') as f:
  for gt_path in gt_paths:
    filename = gt_path.split('/')[-1]
    line = f'{filename} (480,480,3)\n'
    f.write(line)

# Let's make sure the annotations are correct
with open(PATH+train_img_dir+'/training_ann.txt', 'r') as f:
  file_content = f.read()
  print(file_content)

00000.png (480,480,3)
00001.png (480,480,3)
00002.png (480,480,3)
00003.png (480,480,3)
00004.png (480,480,3)
00005.png (480,480,3)
00006.png (480,480,3)
00007.png (480,480,3)
00008.png (480,480,3)
00009.png (480,480,3)
00010.png (480,480,3)
00011.png (480,480,3)
00012.png (480,480,3)
00013.png (480,480,3)
00014.png (480,480,3)
00015.png (480,480,3)
00016.png (480,480,3)
00017.png (480,480,3)
00018.png (480,480,3)
00019.png (480,480,3)
00020.png (480,480,3)
00021.png (480,480,3)
00022.png (480,480,3)
00023.png (480,480,3)
00024.png (480,480,3)
00025.png (480,480,3)
00026.png (480,480,3)
00027.png (480,480,3)
00028.png (480,480,3)
00029.png (480,480,3)
00030.png (480,480,3)
00031.png (480,480,3)
00032.png (480,480,3)
00033.png (480,480,3)
00034.png (480,480,3)
00035.png (480,480,3)
00036.png (480,480,3)
00037.png (480,480,3)
00038.png (480,480,3)
00039.png (480,480,3)
00040.png (480,480,3)
00041.png (480,480,3)
00042.png (480,480,3)
00043.png (480,480,3)
00044.png (480,480,3)
00045.png 

# Configuration

In [ ]:
from mmcv.runner import set_random_seed
from mmcv import Config
# Load the original config
cfg = Config.fromfile('/content/gdrive/MyDrive/Advance Computer Vision/ACV Project 2/srresnet_ffhq_300k.py')

# Dataloaders
cfg.data.train_dataloader = dict(samples_per_gpu=16, drop_last=True, persistent_workers=False)
cfg.data.val_dataloader = dict(samples_per_gpu=1, persistent_workers=False)
cfg.data.test_dataloader = dict(samples_per_gpu=1, persistent_workers=False),

# Training folders
cfg.data.train.dataset.lq_folder = '/content/gdrive/MyDrive/data/FFHQ/train/GT'
cfg.data.train.dataset.gt_folder = '/content/gdrive/MyDrive/data/FFHQ/train/GT'
#cfg.data.train.dataset.ann_file = './content/gdrive/MyDrive/data/FFHQ/train/GT/training_ann.txt'

# Validation folders
cfg.data.val.lq_folder = '/content/gdrive/MyDrive/data/FFHQ/val/LQ'
cfg.data.val.gt_folder = '/content/gdrive/MyDrive/data/FFHQ/val/GT'

# Test folders
cfg.data.test.lq_folder = '/content/gdrive/MyDrive/data/FFHQ/test/LQ'
cfg.data.test.gt_folder = '/content/gdrive/MyDrive/data/FFHQ/val/GT'

# Use the pre-trained SRCNN model
#cfg.load_from = './checkpoint/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth'

# Set up working dir to save files and logs
cfg.work_dir = '/content/gdrive/MyDrive/work_dirs'

# Use smaller batch size for training
cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 0
cfg.data.val_workers_per_gpu = 0

# Set the number of iterations
cfg.total_iters = 138000

# Training scheme
cfg.lr_config = {}
cfg.lr_config.policy = 'Step'
cfg.lr_config.by_epoch = False
cfg.lr_config.step = [100]
cfg.lr_config.gamma = 0.5

# Evaluate every 20 iterations
cfg.evaluation.interval = 2000
if cfg.evaluation.get('gpu_collect', None):
  cfg.evaluation.pop('gpu_collect')

# Save the checkpoints every N iterations
cfg.checkpoint_config.interval = 200

# Print out the log every N iterations
#cfg.log_config.interval = 5
cfg.log_config = dict(
    interval=10, hooks=[dict(type='TextLoggerHook'),
           dict(type='TensorboardLoggerHook')])

cfg.load_from = None
cfg.resume_from = None

# Set seed thus the results are reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpus = 1

print(f'Config:\n{cfg.pretty_text}')  # Show the config

# Training

In [ ]:
import os.path as osp

from mmedit.datasets import build_dataset
from mmedit.models import build_model
from mmedit.apis import train_model
from mmcv.runner import init_dist

import mmcv
import os

# Initialize distributed training (only need to initialize once), comment it if
# have already run this part
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '50297'
init_dist('pytorch', **cfg.dist_params)


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the SRCNN model
model = build_model(
        cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

# Meta information
meta = dict()
if cfg.get('exp_name', None) is None:
    cfg['exp_name'] = osp.splitext(osp.basename(cfg.work_dir))[0]
meta['exp_name'] = cfg.exp_name
meta['mmedit Version'] = mmedit.__version__
meta['seed'] = 0

# Train the model
train_model(model, datasets, cfg, distributed=True, validate=True, meta=meta)

# Testing

In [8]:
from mmedit.apis import single_gpu_test
from mmedit.datasets import build_dataloader
from mmcv.parallel import MMDataParallel


# Build a test dataloader and model
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False,
        persistent_workers=False)
model = MMDataParallel(model, device_ids=[0])

# Perform the test with single gpu. Saving result images by setting
# save_image and save_path arguments. The two arguments will be passed
# to model.forword_test() where images are saved.
# See https://github.com/open-mmlab/mmediting/blob/8b5c0c5f49e60fd6ab0503031b62dee7832faf72/mmedit/models/mattors/indexnet.py#L72.
outputs = single_gpu_test(model, data_loader, save_image=True,
                          save_path='/content/gdrive/MyDrive/data/FFHQ/pred')

# Pop out some unnecessary arguments
eval_config = cfg.evaluation
eval_config.pop('interval')
eval_config.pop('save_image', False)
eval_config.pop('save_path', None)

eval_res = dataset.evaluate(outputs, **eval_config)
print()  # endline of progress bar
#for name, val in eval_res.items():
    #print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 400/400, 1.1 task/s, elapsed: 349s, ETA:     0s


In [26]:
sum(p.numel() for p in model.parameters())

1517571

# Tensorboard

In [ ]:
# load tensorboard in colab
%load_ext tensorboard

# see curves in tensorboard
%tensorboard --logdir '/content/gdrive/MyDrive/work_dirs'